In [3]:
import requests
import os, sys, time
import re
import m3u8
from Crypto.Cipher import AES

## 创建基础文件

In [2]:
file_path = os.getcwd()

In [4]:
final_path = os.path.join(file_path, 'FINAL')
down_path = os.path.join(file_path, 'DOWN')
decrypt_path = os.path.join(file_path, 'DECRYPT')

In [5]:
# 下载那一天的
down_date = 'v.f230'
down_url = 'https://appydydbujb9730.pc.xiaoe-tech.com/live/l_5eeb3665c0bb7_QYY4Trpp'

session = requests.Session()
headers = {
    'Origin': 'https://appydydbujb9730.pc.xiaoe-tech.com',
    'Referer': str(down_url), #获得key必须要
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36'
}

In [6]:
if not os.path.exists(final_path):
    os.makedirs(final_path)

# 切换目录
if not os.path.exists(down_path):
    os.makedirs(down_path)
os.chdir(down_path)

if not os.path.exists(decrypt_path):
    os.makedirs(decrypt_path)

## 解析m3u8文件 

m3u8文件中包含.key文件路径

In [7]:
m3u8_obj = m3u8.load('../{}.m3u8'.format(down_date))
key_uri = m3u8_obj.keys[0].uri
print("key_uri :" + key_uri)

# 下载key 
resp = session.get(key_uri, headers=headers)
print(resp.content)
with open(r'../{}.key'.format(down_date), 'wb') as f:
    f.write(resp.content)

key_uri :https://app.xiaoe-tech.com/get_video_key.php?edk=CiBOzn83jGbyFzuB0cDwQQjuEuLGUPPPUZDpWWX%2FqRhW1BCO08TAChiaoOvUBCokYjRhNjFiNTgtMmVhNy00OWYxLTgwZGMtZTE0NTIyODc5YWIy&fileId=5285890804361143901&keySource=VodBuildInKMS
b'd\xecVi\xff\x8aA\xe2\xe0\xe1\xd0\x88\xfa;\xa2+'


## 下载ts文件

In [8]:
def kaiKeBaBlock(down_url):
    index = re.search(r'start=(\d*)', down_url).group(1) 
    filename = index.zfill(9)

    resp = session.get(down_url, headers=headers)
    with open(r'{}.ts'.format(filename), 'wb') as f:
        f.write(resp.content)
    print('下载完成-----第{}段'.format(index))
    time.sleep(0.02)

In [9]:
base_url = 'https://1252524126.vod2.myqcloud.com/9764a7a5vodtransgzp1252524126/87f23ac25285890804361143901/drm/'

m3u8_obj = m3u8.load('../{}.m3u8'.format(down_date))
uri_list = m3u8_obj.segments
for key in uri_list:
    down_url = base_url + key.uri
    kaiKeBaBlock(down_url)

下载完成-----第0段
下载完成-----第1455888段
下载完成-----第3081536段
下载完成-----第4512784段
下载完成-----第5859248段
下载完成-----第7347280段
下载完成-----第8705024段
下载完成-----第10693136段
下载完成-----第12197520段
下载完成-----第13410880段
下载完成-----第14668800段
下载完成-----第15911296段
下载完成-----第17355328段
下载完成-----第18788272段
下载完成-----第20227424段
下载完成-----第21693456段
下载完成-----第23106480段
下载完成-----第24681184段
下载完成-----第26153792段
下载完成-----第27599328段
下载完成-----第29295472段
下载完成-----第30619184段
下载完成-----第32052320段
下载完成-----第33708416段
下载完成-----第35617184段
下载完成-----第36773584段
下载完成-----第37977360段
下载完成-----第39299008段
下载完成-----第40765984段
下载完成-----第41781008段
下载完成-----第43115248段
下载完成-----第44625088段
下载完成-----第46074960段
下载完成-----第47704560段
下载完成-----第49084496段
下载完成-----第50747744段
下载完成-----第52088752段
下载完成-----第53615888段
下载完成-----第55035296段
下载完成-----第56517120段
下载完成-----第58012864段
下载完成-----第59370416段
下载完成-----第60860320段
下载完成-----第62301920段
下载完成-----第63760432段
下载完成-----第65187168段
下载完成-----第66553376段
下载完成-----第67975984段
下载完成-----第69614416段
下载完成-----第71054128段
下载完成-----第724

### 删除.DS_Store文件

In [10]:
def del_DS_Store(path):
    print(file_path)
    # shell命令查找.DS_Store文件
    os.system(r'find %s -name .DS_Store' %path)
    for root, dirs, files in os.walk(path):
        for name in files:
            if name.find('DS_Store') != -1:
                os.remove(os.path.join(root, name))
                print ('删除文件: ' + os.path.join(root, name))
    print('-'*50)

In [11]:
del_DS_Store(file_path)

/Users/mgbook/Dropbox/--下载单个ts文件
删除文件: /Users/mgbook/Dropbox/--下载单个ts文件/.DS_Store
删除文件: /Users/mgbook/Dropbox/--下载单个ts文件/FINAL/.DS_Store
--------------------------------------------------


## 破解

In [12]:
def decrypt_single_ts(file_name, key, iv):
    raw = open(file_name, 'rb').read()
    data = raw

    cipher = AES.new(key, AES.MODE_CBC, iv=iv)
    plain_data = cipher.decrypt(data)

    open('{0}/{1}'.format(decrypt_path, file_name), 'wb').write(plain_data)
    print('file: ' + file_name + '\tsucceed!')

In [13]:
# 读取ts文件夹下所有的ts文件
file_list = os.listdir()
# 对文件进行排序
file_list.sort()
file_sort = [filename for filename in file_list]


m3u8_obj = m3u8.load('../{}.m3u8'.format(down_date))
iv_str = m3u8_obj.keys[0].iv
print(iv_str)

iv = bytes.fromhex(iv_str[2:])
key = open('../{}.key'.format(down_date), 'rb').read()

# 先删除.DS_Store
del_DS_Store(down_date)
# AES-128解密  
for fname in file_list:
    decrypt_single_ts(fname, key, iv)

0x00000000000000000000000000000000
/Users/mgbook/Dropbox/--下载单个ts文件
--------------------------------------------------
file: 000000000.ts	succeed!
file: 001455888.ts	succeed!
file: 003081536.ts	succeed!
file: 004512784.ts	succeed!
file: 005859248.ts	succeed!
file: 007347280.ts	succeed!
file: 008705024.ts	succeed!
file: 010693136.ts	succeed!
file: 012197520.ts	succeed!
file: 013410880.ts	succeed!
file: 014668800.ts	succeed!
file: 015911296.ts	succeed!
file: 017355328.ts	succeed!
file: 018788272.ts	succeed!
file: 020227424.ts	succeed!
file: 021693456.ts	succeed!
file: 023106480.ts	succeed!
file: 024681184.ts	succeed!
file: 026153792.ts	succeed!
file: 027599328.ts	succeed!
file: 029295472.ts	succeed!
file: 030619184.ts	succeed!
file: 032052320.ts	succeed!
file: 033708416.ts	succeed!
file: 035617184.ts	succeed!
file: 036773584.ts	succeed!
file: 037977360.ts	succeed!
file: 039299008.ts	succeed!
file: 040765984.ts	succeed!
file: 041781008.ts	succeed!
file: 043115248.ts	succeed!
file: 0446250

## 合并

In [14]:
# 先删除.DS_Store
del_DS_Store(decrypt_path)

# 指定输出文件名称
output_file = os.path.join(final_path, '{}.mp4'.format(down_date))
# 使用ffmpeg将ts合并为mp4
os.chdir(decrypt_path)
print("正在合并\t{}.ts文件 ··········".format(down_date))
os.system('cat *.ts > %s.ts'%(down_date))
command = 'ffmpeg -i "%s.ts" -acodec copy -vcodec copy -absf aac_adtstoasc %s'%(down_date, output_file)
# 指行命令
os.system(command)
print("最后转换成 {}.mp4\t完成！".format(down_date))

/Users/mgbook/Dropbox/--下载单个ts文件
删除文件: /Users/mgbook/Dropbox/--下载单个ts文件/DECRYPT/.DS_Store
--------------------------------------------------
正在合并	v.f230.ts文件 ··········
最后转换成 v.f230.mp4	完成！


### 清空文件夹，准备下一个

In [15]:
down_list = os.listdir(down_path)
for fname in down_list:
    os.remove(down_path+ '/' +fname)
print(down_path + "，清理完成！")

decrypt_list = os.listdir(decrypt_path)
for fname in decrypt_list:
    os.remove(decrypt_path+ '/' +fname)
print(decrypt_path + "，清理完成！")

base_path = os.path.split(final_path)[0]
m3u_path = os.path.join(base_path, "%s.m3u8"%(down_date))
key_path = os.path.join(base_path, "%s.key"%(down_date))
os.remove(m3u_path)
os.remove(key_path)
print(key_path + "，清理完成！")
print(m3u_path + "，清理完成！")

/Users/mgbook/Dropbox/--下载单个ts文件/DOWN，清理完成！
/Users/mgbook/Dropbox/--下载单个ts文件/DECRYPT，清理完成！
/Users/mgbook/Dropbox/--下载单个ts文件/v.f230.key，清理完成！
/Users/mgbook/Dropbox/--下载单个ts文件/v.f230.m3u8，清理完成！


In [16]:
os.path.split(final_path)[0]

'/Users/mgbook/Dropbox/--下载单个ts文件'

In [17]:
file_path

'/Users/mgbook/Dropbox/--下载单个ts文件'